# ViTalia - Google ViT v/s Swin Transformer

_______________________

In [3]:
import os
import time
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, SwinForImageClassification, SwinConfig
from sklearn.metrics import accuracy_score
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import warnings

# Suppress warnings related to weight initialization
warnings.filterwarnings("ignore")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set dataset path
data_dir = '/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/'

# Define a custom dataset
class MalariaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, class_name in enumerate(["Parasitized", "Uninfected"]):
            class_dir = os.path.join(root_dir, class_name)
            for img_file in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_file))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except (UnidentifiedImageError, FileNotFoundError):
            print(f"Error loading image {img_path}. Skipping.")
            return self.__getitem__((idx + 1) % len(self))  # Skip to next image if loading fails

        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Split dataset into train and test sets
full_dataset = MalariaDataset(data_dir, transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, pin_memory=True)

# Training function
def train_model(model, epochs=3):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    criterion = torch.nn.CrossEntropyLoss()
    
    # Training loop with timing
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {avg_loss:.4f}")

    end_time = time.time()
    training_time = end_time - start_time
    return training_time

# Evaluation function
def evaluate_model(model):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits
            _, predicted = torch.max(outputs, 1)

            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

# Initialize ViT model
print("Initializing ViT model...")
vit_model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=2)
vit_model.classifier = torch.nn.Linear(vit_model.classifier.in_features, 2)

# Train and evaluate ViT
print("\nTraining ViT model...")
vit_time = train_model(vit_model, epochs=5)
vit_accuracy = evaluate_model(vit_model)
print(f"ViT Training Time: {vit_time:.2f} seconds, Accuracy: {vit_accuracy * 100:.2f}%")

# Initialize Swin Transformer model
print("\nInitializing Swin Transformer model...")
swin_model = SwinForImageClassification(SwinConfig(num_labels=2))

# Train and evaluate Swin
print("\nTraining Swin Transformer model...")
swin_time = train_model(swin_model, epochs=5)
swin_accuracy = evaluate_model(swin_model)
print(f"Swin Training Time: {swin_time:.2f} seconds, Accuracy: {swin_accuracy * 100:.2f}%")

# Print comparison
print("\nModel Comparison:")
print(f"ViT Model - Time: {vit_time:.2f}s, Accuracy: {vit_accuracy * 100:.2f}%")
print(f"Swin Model - Time: {swin_time:.2f}s, Accuracy: {swin_accuracy * 100:.2f}%")

Initializing ViT model...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training ViT model...


Epoch [1/5]:  94%|█████████▍| 1300/1378 [07:56<00:27,  2.81it/s, loss=0.02]   

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected/Thumbs.db. Skipping.


...

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized/Thumbs.db. Skipping.


Epoch [5/5]:  40%|████      | 553/1378 [01:44<03:16,  4.20it/s, loss=0.0118] 

Error loading image /kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected/Thumbs.db. Skipping.


Epoch [5/5]: 100%|██████████| 1378/1378 [04:19<00:00,  5.31it/s, loss=0.195]  


Epoch [5/5], Average Loss: 0.1251
Swin Training Time: 1296.47 seconds, Accuracy: 96.03%

Model Comparison:
ViT Model - Time: 2471.08s, Accuracy: 97.51%
Swin Model - Time: 1296.47s, Accuracy: 96.03%


-------------------------------